# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests
import numpy as np

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [2]:
# 1) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'

requests.get(url)
r = requests.get(url)
r.status_code

r.json()
data_json = r.json()
# print(data_json['items'])

df = pd.DataFrame(data_json['items'])
df.head(3)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,10478,Município,Duque de Caxias,3301702,RJ,None,Regularizado,17944.001209/2012-85,2020-04-13T03:00:00Z,Operação contratual interna,Regularização de Dívida - Outros,Instituição Financeira Nacional,Banco do Brasil S/A,Real,1.497683e+08,0,0,29/05/2020
1,47509,Município,Casimiro de Abreu,3301306,RJ,PVL02.002362/2021-58,Deferido,17944.103628/2021-41,2021-12-24T15:46:02Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,2.900000e+07,0,0,28/12/2021
2,42271,Município,Barra do Piraí,3300308,RJ,PVL02.000610/2020-45,Encaminhado à PGFN com manifestação técnica fa...,17944.101579/2020-21,2020-07-21T14:33:51Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,2.990000e+07,1,1,29/07/2020


In [3]:
# 2) Seu código aqui
df['status'].value_counts()
# ou 
df.status.value_counts()
# visualizar apenas os 3 primeiros
df.status.value_counts().head(3)

Deferido        96
Arquivado       53
Regularizado    26
Name: status, dtype: int64

In [4]:
# 3) Seu código aqui
df['data_status'] = df['data_status'].astype('string')
df.dtypes
df['ano'] = df['data_status'].str[6:]
df['ano']

0      2020
1      2021
2      2020
3      2002
4      2008
       ... 
237    2020
238    2019
239    2008
240    2022
241    2023
Name: ano, Length: 242, dtype: string

In [5]:
df.head(1)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status,ano
0,10478,Município,Duque de Caxias,3301702,RJ,None,Regularizado,17944.001209/2012-85,2020-04-13T03:00:00Z,Operação contratual interna,Regularização de Dívida - Outros,Instituição Financeira Nacional,Banco do Brasil S/A,Real,1.497683e+08,0,0,29/05/2020,2020


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [6]:
#1) Seu código aqui
# respondida

In [24]:
# 2) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=MG&tipo_interessado=Estado'
r = requests.get(url)
resposta = r.json()
df = pd.DataFrame(resposta['items'])

filtro = df.loc[df['status'] == 'Arquivado por decurso de prazo']
filtro.status.value_counts()

Arquivado por decurso de prazo    1
Name: status, dtype: int64

In [55]:
# 3) Seu código aqui
r = requests.get('https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=BA&tipo_interessado=Munic%C3%ADpio')
resposta = r.json()
df = pd.DataFrame(resposta['items'])

deferido = df[df['status'] == 'Deferido']
deferido[['interessado', 'status']].value_counts()
#df.groupby('interessado')['status'].count()
#df[['status', 'interessado']].value_counts()


interessado             status  
Luís Eduardo Magalhães  Deferido    15
Vitória da Conquista    Deferido    11
Camaçari                Deferido    10
Barreiras               Deferido    10
Lauro de Freitas        Deferido    10
                                    ..
Malhada                 Deferido     1
Licínio de Almeida      Deferido     1
Lençóis                 Deferido     1
Cardeal da Silva        Deferido     1
Xique-Xique             Deferido     1
Length: 179, dtype: int64

In [58]:
# 4) Seu código aqui
r = requests.get('https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=BA&tipo_interessado=Estado')
resposta = r.json()

df = pd.DataFrame(resposta['items'])
df.to_csv('baseBahia.csv')